<a href="https://colab.research.google.com/github/carloscl03/tareasMateML/blob/main/Notebooks/Chap12/12_4_Decoding_Strategies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Notebook 12.4: Decoding strategies**

This practical investigates neural decoding from transformer models.  

Work through the cells below, running each cell in turn. In various places you will see the words "TO DO". Follow the instructions at these places and make predictions about what is going to happen or write code to complete the functions.

Contact me at udlbookmail@gmail.com if you find any mistakes or have any suggestions.

In [1]:
!pip install transformers

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, set_seed
import torch
import torch.nn.functional as F
import numpy as np

In [3]:
# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

# Decoding from GPT2

This tutorial investigates how to use GPT2 (the forerunner of GPT3) to generate text.  There are a number of ways to do this that trade-off the realism of the text against the amount of variation.

At every stage, GPT2 takes an input string and returns a probability for each of the possible subsequent tokens.  We can choose what to do with these probability.  We could always *greedily choose* the most likely next token, or we could draw a *sample* randomly according to the probabilities.  There are also intermediate strategies such as *top-k sampling* and *nucleus sampling*, that have some controlled randomness.

We'll also investigate *beam search* -- the idea is that rather than greedily take the next best token at each stage, we maintain a set of hypotheses  (beams)as we add each subsequent token and return the most likely overall hypothesis.  This is not necessarily the same result we get from greedily choosing the next token.

First, let's investigate the token themselves.  The code below prints out the vocabulary size and shows 20 random tokens.  

In [4]:
np.random.seed(1)
print("Number of tokens in dictionary = %d"%(tokenizer.vocab_size))
for i in range(20):
  index = np.random.randint(tokenizer.vocab_size)
  print("Token: %d "%(index)+tokenizer.decode(torch.tensor(index), skip_special_tokens=True))


Number of tokens in dictionary = 50257
Token: 33003  Mormons
Token: 12172  cam
Token: 5192  trig
Token: 32511 ojure
Token: 50057  gist
Token: 43723  Petition
Token: 7813  sin
Token: 21440  Witness
Token: 32912  Remy
Token: 20609 isure
Token: 49100  creeps
Token: 7751  fasc
Token: 43757  Alc
Token: 31228  messenger
Token: 36230  SYSTEM
Token: 32025  precipitation
Token: 21758  cores
Token: 45413  Forestry
Token: 35730  guru
Token: 8444  Disc


# Sampling

Each time we run GPT2 it will take in a set of tokens, and return a probability over each of the possible next tokens.  The simplest thing we could do is to just draw a sample from this probability distribution each time.

In [6]:
def sample_next_token(input_tokens, model, tokenizer):
  # Run model to get prediction over next output
  outputs = model(input_ids = input_tokens['input_ids'], attention_mask = input_tokens['attention_mask'])
  # Find prediction
  prob_over_tokens = F.softmax(outputs.logits, dim=-1).detach().numpy()[0,-1]
  # TODO Draw a random token according to the probabilities
  # next_token should be an array with an sole integer in it (as below)
  # Use:  https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html
  # Replace this line
  next_token = [np.random.choice(len(prob_over_tokens), p=prob_over_tokens)]

  # Append token to sentence
  output_tokens = input_tokens
  output_tokens["input_ids"] = torch.cat((output_tokens['input_ids'],torch.tensor([next_token])),dim=1)
  output_tokens['attention_mask'] = torch.cat((output_tokens['attention_mask'],torch.tensor([[1]])),dim=1)
  output_tokens['last_token_prob'] = prob_over_tokens[next_token]

  return output_tokens

In [7]:
# Expected output:
# "The best thing about Bath is that they don't even change or shrink anymore."

set_seed(0)
input_txt = "The best thing about Bath is"
input_tokens = tokenizer(input_txt, return_tensors='pt')
for i in range(10):
    input_tokens = sample_next_token(input_tokens, model, tokenizer)
    print(tokenizer.decode(input_tokens["input_ids"][0], skip_special_tokens=True))



The best thing about Bath is that
The best thing about Bath is that they
The best thing about Bath is that they don
The best thing about Bath is that they don't
The best thing about Bath is that they don't even
The best thing about Bath is that they don't even change
The best thing about Bath is that they don't even change or
The best thing about Bath is that they don't even change or shrink
The best thing about Bath is that they don't even change or shrink anymore
The best thing about Bath is that they don't even change or shrink anymore.


In [10]:
# TODO Modify the code below by changing the number of tokens generated and the initial sentence
# to get a feel for how well this works.  Since I didn't reset the seed, it will give a different
# answer every time that you run it.

# TODO Experiment with changing this line:
input_txt = "Artificial intelligence is transforming"
input_tokens = tokenizer(input_txt, return_tensors='pt')
# TODO Experiment with changing this line:
for i in range(15):  # Generate 15 tokens
    input_tokens = sample_next_token(input_tokens, model, tokenizer)
    print(tokenizer.decode(input_tokens['input_ids'][0], skip_special_tokens=True))

Artificial intelligence is transforming education
Artificial intelligence is transforming education and
Artificial intelligence is transforming education and raw
Artificial intelligence is transforming education and raw material
Artificial intelligence is transforming education and raw material production
Artificial intelligence is transforming education and raw material production,
Artificial intelligence is transforming education and raw material production, transforming
Artificial intelligence is transforming education and raw material production, transforming user
Artificial intelligence is transforming education and raw material production, transforming user technology
Artificial intelligence is transforming education and raw material production, transforming user technology,
Artificial intelligence is transforming education and raw material production, transforming user technology, transforming
Artificial intelligence is transforming education and raw material production, transfo

# Greedy token selection

You probably (correctly) got the impression that the text from pure sampling of the probability model can be kind of random.  How about if we choose most likely token at each step?


In [11]:
def get_best_next_token(input_tokens, model, tokenizer):
  # Run model to get prediction over next output
  outputs = model(input_ids = input_tokens['input_ids'], attention_mask = input_tokens['attention_mask'])
  # Find prediction
  prob_over_tokens = F.softmax(outputs.logits, dim=-1).detach().numpy()[0,-1]

  # TODO -- find the token index with the maximum probability
  # It should be returns as a list (i.e., put squared brackets around it)
  # Use https://numpy.org/doc/stable/reference/generated/numpy.argmax.html
  # Replace this line
  next_token = [np.argmax(prob_over_tokens)]


  # Append token to sentence
  output_tokens = input_tokens
  output_tokens["input_ids"] = torch.cat((output_tokens['input_ids'],torch.tensor([next_token])),dim=1)
  output_tokens['attention_mask'] = torch.cat((output_tokens['attention_mask'],torch.tensor([[1]])),dim=1)
  output_tokens['last_token_prob'] = prob_over_tokens[next_token]
  return output_tokens

In [13]:
# Expected output:
# The best thing about Bath is that it's a place where you can go to
set_seed(0)
input_txt = "The best thing about Bath is"
input_tokens = tokenizer(input_txt, return_tensors='pt')
for i in range(10):
    input_tokens = get_best_next_token(input_tokens, model, tokenizer)
    print(tokenizer.decode(input_tokens["input_ids"][0], skip_special_tokens=True))

The best thing about Bath is that
The best thing about Bath is that it
The best thing about Bath is that it's
The best thing about Bath is that it's a
The best thing about Bath is that it's a place
The best thing about Bath is that it's a place where
The best thing about Bath is that it's a place where you
The best thing about Bath is that it's a place where you can
The best thing about Bath is that it's a place where you can go
The best thing about Bath is that it's a place where you can go to


In [16]:
# TODO Modify the code below by changing the number of tokens generated and the initial sentence
# to get a feel for how well this works.


# TODO Experiment with changing this line:
input_txt = "In the world of science, it is known that"
input_tokens = tokenizer(input_txt, return_tensors='pt')
# TODO Experiment with changing this line:
for i in range(12):
    input_tokens = get_best_next_token(input_tokens, model, tokenizer)
    print(tokenizer.decode(input_tokens["input_ids"][0], skip_special_tokens=True))

In the world of science, it is known that the
In the world of science, it is known that the most
In the world of science, it is known that the most important
In the world of science, it is known that the most important thing
In the world of science, it is known that the most important thing to
In the world of science, it is known that the most important thing to do
In the world of science, it is known that the most important thing to do is
In the world of science, it is known that the most important thing to do is to
In the world of science, it is known that the most important thing to do is to understand
In the world of science, it is known that the most important thing to do is to understand the
In the world of science, it is known that the most important thing to do is to understand the world
In the world of science, it is known that the most important thing to do is to understand the world around


# Top-K sampling

You probably noticed that the greedy strategy produces quite realistic text, but it's kind of boring.  It produces generic answers.  Also, if this was a chatbot, then we wouldn't necessarily want it to produce the same answer to a question each time.  

Top-K sampling is a compromise strategy that samples randomly from the top K most probable tokens.  We could just choose them with a uniform distribution, or (as here) we could sample them according to their original probabilities.

In [17]:
def get_top_k_token(input_tokens, model, tokenizer, k=20):
  # Run model to get prediction over next output
  outputs = model(input_ids = input_tokens['input_ids'], attention_mask = input_tokens['attention_mask'])
  # Find prediction
  prob_over_tokens = F.softmax(outputs.logits, dim=-1).detach().numpy()[0,-1]

  # Draw a sample from the top K most likely tokens.
  # Take copy of the probabilities and sort from largest to smallest (use np.sort)
  # TODO -- replace this line
  sorted_prob_over_tokens = np.sort(prob_over_tokens)[::-1]

  # Find the probability at the k'th position
  # TODO -- replace this line
  kth_prob_value = sorted_prob_over_tokens[k-1]

  # Set all probabilities below this value to zero
  prob_over_tokens[prob_over_tokens<kth_prob_value] = 0

  # Renormalize the probabilities so that they sum to one
  # TODO -- replace this line
  prob_over_tokens = prob_over_tokens / prob_over_tokens.sum()


  # Draw random token
  next_token = np.random.choice(len(prob_over_tokens), 1, replace=False, p=prob_over_tokens)

  # Append token to sentence
  output_tokens = input_tokens
  output_tokens["input_ids"] = torch.cat((output_tokens['input_ids'],torch.tensor([next_token])),dim=1)
  output_tokens['attention_mask'] = torch.cat((output_tokens['attention_mask'],torch.tensor([[1]])),dim=1)
  output_tokens['last_token_prob'] = prob_over_tokens[next_token]
  return output_tokens

In [18]:
# Expected output:
# The best thing about Bath is that you get to see all the beautiful faces of

set_seed(0)
input_txt = "The best thing about Bath is"
input_tokens = tokenizer(input_txt, return_tensors='pt')
for i in range(10):
    input_tokens = get_top_k_token(input_tokens, model, tokenizer, k=10)
    print(tokenizer.decode(input_tokens["input_ids"][0], skip_special_tokens=True))

<ipython-input-17-fb90990fba34>:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  output_tokens["input_ids"] = torch.cat((output_tokens['input_ids'],torch.tensor([next_token])),dim=1)


The best thing about Bath is that
The best thing about Bath is that you
The best thing about Bath is that you get
The best thing about Bath is that you get to
The best thing about Bath is that you get to see
The best thing about Bath is that you get to see all
The best thing about Bath is that you get to see all the
The best thing about Bath is that you get to see all the beautiful
The best thing about Bath is that you get to see all the beautiful faces
The best thing about Bath is that you get to see all the beautiful faces of


In [21]:
# TODO
# Experiment with different values of k
# If you set it to a lower number (say 3) the text will be less random
# If you set it to a higher number (say 5000) the text will be more random

set_seed(0)
input_txt = "The best thing about Bath is"
input_tokens = tokenizer(input_txt, return_tensors='pt')
for i in range(10):
    input_tokens = get_top_k_token(input_tokens, model, tokenizer, k=5000)
    print(tokenizer.decode(input_tokens["input_ids"][0], skip_special_tokens=True))

The best thing about Bath is that
The best thing about Bath is that all
The best thing about Bath is that all you
The best thing about Bath is that all you have
The best thing about Bath is that all you have to
The best thing about Bath is that all you have to do
The best thing about Bath is that all you have to do is
The best thing about Bath is that all you have to do is submit
The best thing about Bath is that all you have to do is submit applications
The best thing about Bath is that all you have to do is submit applications in


# Nucleus sampling

Top-K sampling has the disadvantage that sometimes there are only a few plausible next tokens, and sometimes there are a lot.  How do we adapt to this situation?  One way is to sample from a fixed proportion of the probability mass.  That is we order the tokens in terms of probability and cut off the possibility of sampling when the cumulative sum is greater than a threshold.

This way, we adapt the number of possible tokens that we can choose.

In [22]:
def get_nucleus_sampling_token(input_tokens, model, tokenizer, thresh=0.25):
  # Run model to get prediction over next output
  outputs = model(input_ids = input_tokens['input_ids'], attention_mask = input_tokens['attention_mask'])
  # Find prediction
  prob_over_tokens = F.softmax(outputs.logits, dim=-1).detach().numpy()[0,-1]

  # Find the most likely tokens that make up the first (thresh) of the probability
  # TODO -- sort the probabilities in decreasing order
  # Replace this line
  sorted_probs_decreasing = prob_over_tokens
  # TODO -- compute the cumulative sum of these probabilities
  # Replace this line
  cum_sum_probs = np.cumsum(sorted_probs_decreasing)



  # Find index where that the cumulative sum is greater than the threshold
  thresh_index = np.argmax(cum_sum_probs>thresh)
  print("Choosing from %d tokens"%(thresh_index))
  # TODO:  Find the probability value to threshold
  # Replace this line:
  thresh_prob = sorted_probs_decreasing[thresh_index]


  # Set any probabilities less than this to zero
  prob_over_tokens[prob_over_tokens<thresh_prob] = 0
  # Renormalize
  prob_over_tokens = prob_over_tokens / np.sum(prob_over_tokens)
  # Draw random token
  next_token = np.random.choice(len(prob_over_tokens), 1, replace=False, p=prob_over_tokens)

  # Append token to sentence
  output_tokens = input_tokens
  output_tokens["input_ids"] = torch.cat((output_tokens['input_ids'],torch.tensor([next_token])),dim=1)
  output_tokens['attention_mask'] = torch.cat((output_tokens['attention_mask'],torch.tensor([[1]])),dim=1)
  output_tokens['last_token_prob'] = prob_over_tokens[next_token]
  return output_tokens

In [23]:
# Expected output:
# The best thing about Bath is that it's not a city that has been around
set_seed(0)
input_txt = "The best thing about Bath is"
input_tokens = tokenizer(input_txt, return_tensors='pt')
for i in range(10):
    input_tokens = get_nucleus_sampling_token(input_tokens, model, tokenizer, thresh = 0.2)
    print(tokenizer.decode(input_tokens["input_ids"][0], skip_special_tokens=True))


Choosing from 326 tokens
The best thing about Bath is that
Choosing from 340 tokens
The best thing about Bath is that it
Choosing from 338 tokens
The best thing about Bath is that it's
Choosing from 407 tokens
The best thing about Bath is that it's a
Choosing from 1180 tokens
The best thing about Bath is that it's a place
Choosing from 326 tokens
The best thing about Bath is that it's a place where
Choosing from 345 tokens
The best thing about Bath is that it's a place where you
Choosing from 460 tokens
The best thing about Bath is that it's a place where you can
Choosing from 599 tokens
The best thing about Bath is that it's a place where you can celebrate
Choosing from 290 tokens
The best thing about Bath is that it's a place where you can celebrate your


In [24]:
# TODO -- experiment with setting the threshold probability to larger or smaller values
print("Experiment with thresh = 0.2:")
set_seed(0)
input_txt = "The best thing about Bath is"
input_tokens = tokenizer(input_txt, return_tensors='pt')
for i in range(10):
    input_tokens = get_nucleus_sampling_token(input_tokens, model, tokenizer, thresh=0.2)
    print(tokenizer.decode(input_tokens["input_ids"][0], skip_special_tokens=True))

print("\nExperiment with thresh = 0.1 (less random):")
set_seed(0)
input_txt = "The best thing about Bath is"
input_tokens = tokenizer(input_txt, return_tensors='pt')
for i in range(10):
    input_tokens = get_nucleus_sampling_token(input_tokens, model, tokenizer, thresh=0.1)
    print(tokenizer.decode(input_tokens["input_ids"][0], skip_special_tokens=True))

print("\nExperiment with thresh = 0.5:")
set_seed(0)
input_txt = "The best thing about Bath is"
input_tokens = tokenizer(input_txt, return_tensors='pt')
for i in range(10):
    input_tokens = get_nucleus_sampling_token(input_tokens, model, tokenizer, thresh=0.5)
    print(tokenizer.decode(input_tokens["input_ids"][0], skip_special_tokens=True))

print("\nExperiment with thresh = 0.9 (more random):")
set_seed(0)
input_txt = "The best thing about Bath is"
input_tokens = tokenizer(input_txt, return_tensors='pt')
for i in range(10):
    input_tokens = get_nucleus_sampling_token(input_tokens, model, tokenizer, thresh=0.9)
    print(tokenizer.decode(input_tokens["input_ids"][0], skip_special_tokens=True))

Experiment with thresh = 0.2:
Choosing from 326 tokens
The best thing about Bath is that
Choosing from 340 tokens
The best thing about Bath is that it
Choosing from 338 tokens
The best thing about Bath is that it's
Choosing from 407 tokens
The best thing about Bath is that it's a
Choosing from 1180 tokens
The best thing about Bath is that it's a place
Choosing from 326 tokens
The best thing about Bath is that it's a place where
Choosing from 345 tokens
The best thing about Bath is that it's a place where you
Choosing from 460 tokens
The best thing about Bath is that it's a place where you can
Choosing from 599 tokens
The best thing about Bath is that it's a place where you can celebrate
Choosing from 290 tokens
The best thing about Bath is that it's a place where you can celebrate your

Experiment with thresh = 0.1 (less random):
Choosing from 262 tokens
The best thing about Bath is that
Choosing from 314 tokens
The best thing about Bath is that you
Choosing from 460 tokens
The best th

# Beam search

All of the methods we've seen so far choose the tokens one by one.  But this isn't necessarily sensible.  Even greedily choosing the best token doesn't necessarily retrieve the sequence with the highest probability.  It might be that the most likely token only has very unlikely tokens following it.

Beam search maintains $K$ hypotheses about the best possible continuation.  It starts with the top $K$ continuations.  Then for each of those, it finds the top K continuations, giving $K^2$ hypotheses.  Then it retains just the top $K$ of these so that the number of hypotheses stays the same.

In [25]:
# This routine returns the k'th most likely next token.
# If k =0 then it returns the most likely token, if k=1 it returns the next most likely and so on
# We will need this for beam search
def get_kth_most_likely_token(input_tokens, model, tokenizer, k):
  # Run model to get prediction over next output
  outputs = model(input_ids = input_tokens['input_ids'], attention_mask = input_tokens['attention_mask'])
  # Find prediction
  prob_over_tokens = F.softmax(outputs.logits, dim=-1).detach().numpy()[0,-1]

  # Find the k'th most likely token
  # TODO Sort the probabilities from largest to smallest
  # Replace this line:
  sorted_prob_over_tokens = np.sort(prob_over_tokens)[::-1]
  # TODO Find the k'th sorted probability
  # Replace this line
  kth_prob_value = sorted_prob_over_tokens[k]



  # Find position of this token.
  next_token = np.where(prob_over_tokens == kth_prob_value)[0]

  # Append token to sentence
  output_tokens = input_tokens
  output_tokens["input_ids"] = torch.cat((output_tokens['input_ids'],torch.tensor([next_token])),dim=1)
  output_tokens['attention_mask'] = torch.cat((output_tokens['attention_mask'],torch.tensor([[1]])),dim=1)
  output_tokens['last_token_prob'] = prob_over_tokens[next_token]
  output_tokens['log_prob'] = output_tokens['log_prob'] + np.log(prob_over_tokens[next_token])
  return output_tokens

In [26]:
# We can test this code and see that if we choose the 2nd most likely (K=1) token each time
# then we get much better generation results than if we choose the 2001st most likely token

# Expected output:
# The best thing about Bath is the way you get the most bang outta the
set_seed(0)
input_txt = "The best thing about Bath is"
input_tokens = tokenizer(input_txt, return_tensors='pt')
input_tokens['log_prob'] = 0.0
for i in range(10):
    input_tokens = get_kth_most_likely_token(input_tokens, model, tokenizer, k=1)
    print(tokenizer.decode(input_tokens["input_ids"][0], skip_special_tokens=True))

# Expected output:
# The best thing about Bath is mixed profits partnerships» buy generic+ Honda throttlecont
input_txt = "The best thing about Bath is"
input_tokens = tokenizer(input_txt, return_tensors='pt')
input_tokens['log_prob'] = 0.0
for i in range(10):
    input_tokens = get_kth_most_likely_token(input_tokens, model, tokenizer, k=2000)
    print(tokenizer.decode(input_tokens["input_ids"][0], skip_special_tokens=True))

# TODO -- play around with different values of K
print("\nExperiment with different values of k:")
input_txt = "The best thing about Bath is"
input_tokens = tokenizer(input_txt, return_tensors='pt')
input_tokens['log_prob'] = 0.0
for k in [10, 100, 500]:
    print(f"\nUsing k={k}:")
    for i in range(10):
        input_tokens = get_kth_most_likely_token(input_tokens, model, tokenizer, k=k)
        print(tokenizer.decode(input_tokens["input_ids"][0], skip_special_tokens=True))

The best thing about Bath is the
The best thing about Bath is the way
The best thing about Bath is the way you
The best thing about Bath is the way you get
The best thing about Bath is the way you get the
The best thing about Bath is the way you get the most
The best thing about Bath is the way you get the most bang
The best thing about Bath is the way you get the most bang out
The best thing about Bath is the way you get the most bang outta
The best thing about Bath is the way you get the most bang outta the
The best thing about Bath is mixed
The best thing about Bath is mixed profits
The best thing about Bath is mixed profits partnerships
The best thing about Bath is mixed profits partnerships»
The best thing about Bath is mixed profits partnerships» buy
The best thing about Bath is mixed profits partnerships» buy generic
The best thing about Bath is mixed profits partnerships» buy generic+
The best thing about Bath is mixed profits partnerships» buy generic+ Honda
The best thing abo

In [27]:
# Print out each beam plus the log probability
def print_beams(beams):
  for index,beam in enumerate(beams):
    print("Beam %d, Prob %3.3f: "%(index,beam['log_prob'])+tokenizer.decode(beam["input_ids"][0], skip_special_tokens=True))
  print('---')


# TODO:  Read this code carefully!
def do_beam_search(input_tokens_in, model, tokenizer, n_beam=5, beam_length=10):
  # Store beams in a list
  input_tokens['log_prob'] = 0.0

  # Initialize with n_beam most likely continuations
  beams = [None] * n_beam
  for c_k in range(n_beam):
    beams[c_k] = dict(input_tokens_in)
    beams[c_k] = get_kth_most_likely_token(beams[c_k], model, tokenizer, c_k)

  print_beams(beams)

  # For each token in the sequence we will add
  for c_pos in range(beam_length-1):
    # Now for each beam, we continue it in the most likely ways, making n_beam*n_beam type hypotheses
    beams_all = [None] * (n_beam*n_beam)
    log_probs_all = np.zeros(n_beam*n_beam)
    # For each current hypothesis
    for c_beam in range(n_beam):
      # For each continuation
      for c_k in range(n_beam):
        # Store the continuation and the probability
        beams_all[c_beam * n_beam + c_k] = dict(get_kth_most_likely_token(beams[c_beam], model, tokenizer, c_k))
        log_probs_all[c_beam * n_beam + c_k] = beams_all[c_beam * n_beam + c_k]['log_prob']

    # Keep the best n_beams sequences with the highest probabilities
    sorted_index = np.argsort(np.array(log_probs_all)*-1)
    for c_k in range(n_beam):
      beams[c_k] = dict(beams_all[sorted_index[c_k]])

    # Print the beams
    print_beams(beams)

  return beams[0]

In [28]:
# Expected output:
# The best thing about Bath is that it's a place where you don't have to

set_seed(0)
input_txt = "The best thing about Bath is"
input_tokens = tokenizer(input_txt, return_tensors='pt')

# Now let's call the beam search
# It takes a while as it has to run the model multiple times to add a token
n_beams = 5
best_beam = do_beam_search(input_tokens,model,tokenizer)
print("Beam search result:")
print(tokenizer.decode(best_beam["input_ids"][0], skip_special_tokens=True))

# You should see that the best answer is not the same as the greedy solution we found above


<ipython-input-27-a3dcefd79d9e>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print("Beam %d, Prob %3.3f: "%(index,beam['log_prob'])+tokenizer.decode(beam["input_ids"][0], skip_special_tokens=True))
<ipython-input-27-a3dcefd79d9e>:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  log_probs_all[c_beam * n_beam + c_k] = beams_all[c_beam * n_beam + c_k]['log_prob']


Beam 0, Prob -0.727: The best thing about Bath is that
Beam 1, Prob -2.161: The best thing about Bath is the
Beam 2, Prob -3.177: The best thing about Bath is it
Beam 3, Prob -3.468: The best thing about Bath is how
Beam 4, Prob -3.536: The best thing about Bath is you
---
Beam 0, Prob -1.899: The best thing about Bath is that it
Beam 1, Prob -3.931: The best thing about Bath is it's
Beam 2, Prob -4.079: The best thing about Bath is that it is
Beam 3, Prob -4.433: The best thing about Bath is the fact
Beam 4, Prob -4.553: The best thing about Bath is you can
---
Beam 0, Prob -2.740: The best thing about Bath is that it's
Beam 1, Prob -4.657: The best thing about Bath is the fact that
Beam 2, Prob -5.331: The best thing about Bath is that it's not
Beam 3, Prob -6.227: The best thing about Bath is it's a
Beam 4, Prob -6.264: The best thing about Bath is that it is a
---
Beam 0, Prob -4.938: The best thing about Bath is that it's a
Beam 1, Prob -6.012: The best thing about Bath is the fac

You can read about more decoding strategies in this blog (which uses a recursive neural network, not a transformer, but the principles are the same).

https://www.borealisai.com/research-blogs/tutorial-6-neural-natural-language-generation-decoding-algorithms/

You can also look at other possible language models via hugging face:

https://huggingface.co/docs/transformers/v4.25.1/en/model_summary#decoders-or-autoregressive-models
